In [8]:
# --- Inference helper (binary & multiclass) ---
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms, models
from typing import List

# Device configuration
def device_auto() -> torch.device:
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

device = device_auto()

# ImageNet normalization
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# Load the saved model
def load_trained_model(model_path: str, num_classes: int) -> nn.Module:
    """Load the trained model from checkpoint"""
    # Rebuild the model architecture
    model = models.resnet18(weights=None)
    in_feats = model.fc.in_features
    
    # Recreate the same head structure used during training
    if num_classes == 2:
        model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, 1))
    else:
        model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, num_classes))
    
    # Load the saved weights
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint)
    model = model.to(device)
    model.eval()
    
    return model

# Transformation for inference
infer_tfms = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure image is 224x224
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

def predict_image(model: nn.Module, img_path: str, classes: List[str]) -> dict:
    """Predict class for a single image using the trained model"""
    model.eval()
    
    # Load and preprocess image
    img = Image.open(img_path).convert("RGB")
    x = infer_tfms(img).unsqueeze(0).to(device)
    
    # Prediction
    with torch.no_grad():
        logits = model(x)
        
        if len(classes) == 2:
            # Binary classification
            p = torch.sigmoid(logits).item()
            pred_idx = int(p >= 0.5)
            return {
                "pred_idx": pred_idx, 
                "pred_class": classes[pred_idx], 
                "prob_positive": float(p),
                "prob_negative": float(1 - p)
            }
        else:
            # Multi-class classification
            probs = torch.softmax(logits, dim=1).squeeze(0)
            pred_idx = int(probs.argmax().item())
            prob_dict = {classes[i]: float(probs[i]) for i in range(len(classes))}
            
            return {
                "pred_idx": pred_idx, 
                "pred_class": classes[pred_idx], 
                "probabilities": prob_dict,
                "confidence": float(probs[pred_idx])
            }

In [9]:
MODEL_PATH = "outputs/resnet18_finetuned_with_head.pt"
CLASSES = ['Negative', 'Positive']
NUM_CLASSES = len(CLASSES)

# Load model once
model = load_trained_model(MODEL_PATH, NUM_CLASSES)

# Predict on an image
result = predict_image(model, "test1/7.jpg", CLASSES)
print(result)

{'pred_idx': 1, 'pred_class': 'Positive', 'prob_positive': 1.0, 'prob_negative': 0.0}


In [ ]:
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet18

# --- Quantum Imports ---
import pennylane as qml
from pennylane import numpy as pnp

# --- 1. Define Model Architecture & Constants ---

# Model structure constants
n_qubits = 4
n_layers = 6

# Preprocessing constants
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# PennyLane device
dev = qml.device("default.qubit", wires=n_qubits)

# --- Class definition for L512to4 ---
class L512to4(nn.Module):
    def __init__(self, in_dim=512, hidden_dim=4):
        super().__init__()
        self.fc = nn.Linear(in_dim, hidden_dim)
        self.act = nn.Tanh()
    def forward(self, z):
        return self.act(self.fc(z))

# --- Quantum block definitions---
def entangle_ladder():
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

@qml.qnode(dev, interface="torch")
def quantum_block(x, weights):
    for q in range(n_qubits):
        qml.Hadamard(wires=q)
        qml.RY(pnp.pi * x[q] / 2.0, wires=q)
    for l in range(n_layers):
        for q in range(n_qubits):
            qml.RY(weights[l, q], wires=q)
        entangle_ladder()
    return [qml.expval(qml.PauliZ(q)) for q in range(n_qubits)]

# --- Class definition for QuantumLayer---
class QuantumLayer(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial value doesn't matter, it will be overwritten
        w0 = 0.01 * torch.randn(n_layers, n_qubits)
        self.weights = nn.Parameter(w0)
    def forward(self, x4_batch):
        outs = []
        for i in range(x4_batch.shape[0]):
            y = quantum_block(x4_batch[i], self.weights)
            y = torch.stack(y)
            outs.append(y)
        zq = torch.stack(outs, dim=0)
        zq = zq.to(torch.float32)
        return zq

# --- Class definition for L4to2---
class L4to2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(4, 2)
    def forward(self, z4):
        return self.fc(z4)

# --- Class definition for HybridModel---
class HybridModel(nn.Module):
    def __init__(self, backbone, proj, q_layer, head):
        super().__init__()
        self.backbone = backbone
        self.proj = proj
        self.q_layer = q_layer
        self.head = head
    def forward(self, imgs):
        with torch.no_grad():
            z512 = self.backbone(imgs)
        x4 = self.proj(z512)
        zq = self.q_layer(x4)
        logits = self.head(zq)
        return logits

# --- 2. Define Inference Function---

# Preprocessing transforms
infer_tf = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Prediction function
@torch.no_grad()
def predict_one_with_model(img_path: str, model_for_infer: torch.nn.Module, class_names=None):
    img = Image.open(img_path).convert("RGB")
    x = infer_tf(img).unsqueeze(0).to(device)

    logits = model_for_infer(x)

    # Use the class names from the checkpoint if available, otherwise default
    if class_names is None:
        class_names = [f"class{i}" for i in range(logits.shape[1])]

    # binary or multi-class friendly
    if logits.shape[1] == 1:
        p1 = torch.sigmoid(logits[:, 0])
        probs = torch.stack([1 - p1, p1], dim=1)
    else:
        probs = F.softmax(logits, dim=1)

    probs = probs.squeeze(0).cpu()
    pred_idx = int(probs.argmax().item())

    print(f"Image: {img_path}")
    print(f"Prediction: {class_names[pred_idx]} (idx={pred_idx})")
    for i, p in enumerate(probs.tolist()):
        print(f"  {class_names[i]:>12s}: {p:.4f}")

    return {"label": class_names[pred_idx], "index": pred_idx, "probs": probs.tolist()}

In [11]:
CHECKPOINT_PATH = "artifacts/hybrid_qml_best.pt"
TEST_IMG = "test1/1.jpg"

In [ ]:

print(f"Using device: {device}")
if not Path(CHECKPOINT_PATH).exists():
    print(f"Error: Checkpoint file not found at {CHECKPOINT_PATH}")
    print("Please run the training notebook first to create this file.")
elif not Path(TEST_IMG).exists():
    print(f"Error: Test image not found at {TEST_IMG}")
else:
    try:
        # 1. Rebuild the SAME hybrid architecture
        print("Rebuilding hybrid model architecture...")
        backbone = resnet18(weights=None)
        backbone.fc = torch.nn.Identity() # Remove classifier head
        
        proj = L512to4(in_dim=512, hidden_dim=n_qubits)
        q_layer = QuantumLayer()
        head = L4to2() # Outputs 2 classes
        
        model_inf = HybridModel(backbone, proj, q_layer, head).to(device)
        # 2. Load the saved weights
        print(f"Loading weights from: {CHECKPOINT_PATH}")
        
        # Added weights_only=False because the file is trusted
        # and contains more than just tensors.
        ckpt = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=False)
        
        model_inf.load_state_dict(ckpt["state_dict"])
        model_inf.eval()
        print("Model loaded successfully.")
        # 3. Get class names from the checkpoint metadata
        class_names = ckpt.get("meta", {}).get("class_names", ["negative", "positive"])
        # 4. Predict the image
        result = predict_one_with_model(TEST_IMG, model_inf, class_names=class_names)
        
    except Exception as e:
        print(f"An error occurred during model loading or prediction: {e}")
        import traceback
        traceback.print_exc()

Using device: cuda
Rebuilding hybrid model architecture...
Loading weights from: artifacts/hybrid_qml_best.pt
Model loaded successfully.
Image: test1/1.jpg
Prediction: negative (idx=0)
      negative: 0.7607
      positive: 0.2393


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from PIL import Image
from torchvision import transforms, models
from typing import List
import numpy as np

# --- Quantum Imports ---
try:
    import pennylane as qml
    from pennylane import numpy as pnp
except ImportError:
    print("Warning: PennyLane not found. Quantum model functionality will fail.")
    qml = None
    pnp = None

# ======================================================================
# --- 1. Common Definitions
# ======================================================================

# --- Device and Constants ---
def device_auto() -> torch.device:
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

device = device_auto()

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# --- Paths ---
CLASSICAL_MODEL_PATH = "outputs/resnet18_finetuned_with_head.pt"
QUANTUM_MODEL_PATH = "artifacts/hybrid_qml_best.pt"
TEST_DIR = Path("final_test_full")

# --- Class Lists ---
CLASSES_CLASSICAL = ['Negative', 'Positive']
CLASSES_QUANTUM = ['negative', 'positive'] # Note the lowercase

# ======================================================================
# --- 2. Classical Model Definitions
# ======================================================================

def load_classical_model(model_path: str, num_classes: int) -> nn.Module:
    """Load the trained classical model from checkpoint"""
    model = models.resnet18(weights=None)
    in_feats = model.fc.in_features
    
    if num_classes == 2:
        model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, 1))
    else:
        model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, num_classes))
    
    try:
        # Try loading as a full state_dict first
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint)
    except Exception:
        # If it fails, assume it's a dict and try to get 'state_dict'
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint.get('state_dict', checkpoint))
        
    model = model.to(device)
    model.eval()
    return model

classical_infer_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

def predict_classical(model: nn.Module, img_path: str, classes: List[str]) -> dict:
    """Predict class for a single image using the classical model"""
    model.eval()
    img = Image.open(img_path).convert("RGB")
    x = classical_infer_tfms(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        logits = model(x)
        p = torch.sigmoid(logits).item()
        pred_idx = int(p >= 0.5)
        return {
            "pred_idx": pred_idx, 
            "pred_class": classes[pred_idx], 
            "prob_positive": float(p),
            "prob_negative": float(1 - p)
        }

# ======================================================================
# --- 3. Quantum Model Definitions
# ======================================================================

if qml:
    # Model structure constants
    n_qubits = 4
    n_layers = 6

    # PennyLane device
    dev = qml.device("default.qubit", wires=n_qubits)

    class L512to4(nn.Module):
        def __init__(self, in_dim=512, hidden_dim=4):
            super().__init__()
            self.fc = nn.Linear(in_dim, hidden_dim)
            self.act = nn.Tanh()
        def forward(self, z):
            return self.act(self.fc(z))

    def entangle_ladder():
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])

    @qml.qnode(dev, interface="torch")
    def quantum_block(x, weights):
        for q in range(n_qubits):
            qml.Hadamard(wires=q)
            qml.RY(pnp.pi * x[q] / 2.0, wires=q)
        for l in range(n_layers):
            for q in range(n_qubits):
                qml.RY(weights[l, q], wires=q)
            entangle_ladder()
        return [qml.expval(qml.PauliZ(q)) for q in range(n_qubits)]

    class QuantumLayer(nn.Module):
        def __init__(self):
            super().__init__()
            w0 = 0.01 * torch.randn(n_layers, n_qubits)
            self.weights = nn.Parameter(w0)
        def forward(self, x4_batch):
            outs = []
            for i in range(x4_batch.shape[0]):
                y = quantum_block(x4_batch[i], self.weights)
                y = torch.stack(y)
                outs.append(y)
            zq = torch.stack(outs, dim=0)
            return zq.to(torch.float32)

    class L4to2(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc = nn.Linear(4, 2)
        def forward(self, z4):
            return self.fc(z4)

    class HybridModel(nn.Module):
        def __init__(self, backbone, proj, q_layer, head):
            super().__init__()
            self.backbone = backbone
            self.proj = proj
            self.q_layer = q_layer
            self.head = head
        def forward(self, imgs):
            with torch.no_grad():
                z512 = self.backbone(imgs)
            x4 = self.proj(z512)
            zq = self.q_layer(x4)
            logits = self.head(zq)
            return logits

    def load_quantum_model(model_path: str) -> nn.Module:
        """Build and load the hybrid quantum model"""
        backbone = models.resnet18(weights=None)
        backbone.fc = torch.nn.Identity()
        proj = L512to4(in_dim=512, hidden_dim=n_qubits)
        q_layer = QuantumLayer()
        head = L4to2()
        
        model_inf = HybridModel(backbone, proj, q_layer, head).to(device)
        
        # Load the checkpoint (which is a dict)
        ckpt = torch.load(model_path, map_location=device, weights_only=False)
        model_inf.load_state_dict(ckpt["state_dict"])
        model_inf.eval()
        
        # Also return the class names from metadata
        class_names = ckpt.get("meta", {}).get("class_names", CLASSES_QUANTUM)
        return model_inf, class_names

    quantum_infer_tfms = transforms.Compose([
        transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
    ])

    @torch.no_grad()
    def predict_quantum(model: nn.Module, img_path: str, class_names: List[str]) -> dict:
        """Predict class for a single image using the quantum model"""
        img = Image.open(img_path).convert("RGB")
        x = quantum_infer_tfms(img).unsqueeze(0).to(device)

        logits = model(x)
        probs = F.softmax(logits, dim=1).squeeze(0).cpu()
        pred_idx = int(probs.argmax().item())

        return {
            "pred_idx": pred_idx,
            "label": class_names[pred_idx],
            "probs": probs.tolist(), # [prob_neg, prob_pos]
            "prob_negative": probs[0],
            "prob_positive": probs[1],
        }
else:
    print("Skipping Quantum Model definitions.")


# ======================================================================
# --- 4. Main Evaluation Function
# ======================================================================

def run_evaluation():
    print(f"Using device: {device}")
    
    # --- 1. Load Models ---
    print(f"Loading classical model from: {CLASSICAL_MODEL_PATH}")
    try:
        classical_model = load_classical_model(CLASSICAL_MODEL_PATH, len(CLASSES_CLASSICAL))
    except FileNotFoundError:
        print(f"Error: Classical model file not found at {CLASSICAL_MODEL_PATH}")
        return
    
    print(f"Loading quantum model from: {QUANTUM_MODEL_PATH}")
    try:
        quantum_model, quantum_classes = load_quantum_model(QUANTUM_MODEL_PATH)
        # Ensure quantum classes are lowercase for comparison
        quantum_classes = [c.lower() for c in quantum_classes]
    except FileNotFoundError:
        print(f"Error: Quantum model file not found at {QUANTUM_MODEL_PATH}")
        return
    except NameError:
        print("Error: Quantum model classes are not defined. Was PennyLane imported?")
        return

    # --- 2. Find Test Images ---
    print(f"Scanning for images in: {TEST_DIR}")
    if not TEST_DIR.exists():
        print(f"Error: Test directory not found at {TEST_DIR}")
        return
        
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    image_files = [
        p for p in TEST_DIR.rglob('*') 
        if p.suffix.lower() in image_extensions
    ]
    
    if not image_files:
        print(f"Error: No images found in {TEST_DIR}")
        return
    
    print(f"Found {len(image_files)} images to test.")

    # --- 3. Run Predictions and Collect Results ---
    all_results = []
    print("Running predictions...")
    
    for img_path in image_files:
        # Get ground truth label from parent folder name (e.g., 'positive' or 'negative')
        gt_label = img_path.parent.name.lower()
        
        # Run Classical Prediction
        c_res = predict_classical(classical_model, str(img_path), CLASSES_CLASSICAL)
        c_pred_label = c_res['pred_class'].lower()
        c_correct = (c_pred_label == gt_label)
        c_prob_pos = c_res['prob_positive']
        c_confidence = c_prob_pos if c_pred_label == 'positive' else (1 - c_prob_pos)
        
        # Run Quantum Prediction
        q_res = predict_quantum(quantum_model, str(img_path), quantum_classes)
        q_pred_label = q_res['label'].lower()
        q_correct = (q_pred_label == gt_label)
        q_prob_pos = q_res['prob_positive']
        q_confidence = q_prob_pos if q_pred_label == 'positive' else (1 - q_prob_pos)

        all_results.append({
            "path": str(img_path.relative_to(TEST_DIR)),
            "gt": gt_label,
            "c_pred": c_pred_label,
            "c_correct": c_correct,
            "c_prob_pos": c_prob_pos,
            "c_confidence": c_confidence,
            "q_pred": q_pred_label,
            "q_correct": q_correct,
            "q_prob_pos": q_prob_pos,
            "q_confidence": q_confidence,
        })

    # --- 4. Analyze and Report ---
    total = len(all_results)
    
    # Overall Accuracy
    c_total_correct = sum(1 for r in all_results if r['c_correct'])
    q_total_correct = sum(1 for r in all_results if r['q_correct'])
    
    print("\n" + "="*30)
    print("  EVALUATION REPORT")
    print("="*30)
    
    print(f"\n--- 1. Overall Accuracy ---")
    print(f"Classical: {c_total_correct}/{total}  ({c_total_correct/total:.2%})")
    print(f"Quantum:   {q_total_correct}/{total}  ({q_total_correct/total:.2%})")

    # Probability / Confidence Analysis
    c_conf_correct = [r['c_confidence'] for r in all_results if r['c_correct']]
    c_conf_wrong = [r['c_confidence'] for r in all_results if not r['c_correct']]
    q_conf_correct = [r['q_confidence'] for r in all_results if r['q_correct']]
    q_conf_wrong = [r['q_confidence'] for r in all_results if not r['q_correct']]

    print(f"\n--- 2. Average Confidence ---")
    print(f"  (Probability assigned to the *predicted* class)")
    print(f"Classical (Correct): {np.mean(c_conf_correct):.4f}" if c_conf_correct else "Classical (Correct): N/A")
    print(f"Classical (Wrong):   {np.mean(c_conf_wrong):.4f}" if c_conf_wrong else "Classical (Wrong):   N/A")
    print(f"Quantum (Correct):   {np.mean(q_conf_correct):.4f}" if q_conf_correct else "Quantum (Correct):   N/A")
    print(f"Quantum (Wrong):     {np.mean(q_conf_wrong):.4f}" if q_conf_wrong else "Quantum (Wrong):     N/A")

    # Probability Margin Analysis (Difference from 0.5)
    c_margins = [abs(r['c_prob_pos'] - 0.5) for r in all_results]
    q_margins = [abs(r['q_prob_pos'] - 0.5) for r in all_results]

    print(f"\n--- 3. Average Probability Margin ---")
    print(f"  (Average distance from 0.5, a measure of decisiveness)")
    print(f"Classical: {np.mean(c_margins):.4f}")
    print(f"Quantum:   {np.mean(q_margins):.4f}")

    # Discrepancy Report
    c_right_q_wrong = [r for r in all_results if r['c_correct'] and not r['q_correct']]
    q_right_c_wrong = [r for r in all_results if not r['c_correct'] and r['q_correct']]
    both_wrong = [r for r in all_results if not r['c_correct'] and not r['q_correct']]

    print(f"\n--- 4. Discrepancy Report ---")
    
    print(f"\nClassical CORRECT, Quantum WRONG ({len(c_right_q_wrong)}):")
    for r in c_right_q_wrong:
        print(f"  - {r['path']} (GT: {r['gt']})")
        print(f"    Classical predicted: {r['c_pred']} ({r['c_confidence']:.3f})")
        print(f"    Quantum predicted:   {r['q_pred']} ({r['q_confidence']:.3f})")

    print(f"\nQuantum CORRECT, Classical WRONG ({len(q_right_c_wrong)}):")
    for r in q_right_c_wrong:
        print(f"  - {r['path']} (GT: {r['gt']})")
        print(f"    Classical predicted: {r['c_pred']} ({r['c_confidence']:.3f})")
        print(f"    Quantum predicted:   {r['q_pred']} ({r['q_confidence']:.3f})")

    print(f"\nBoth WRONG ({len(both_wrong)}):")
    for r in both_wrong:
        print(f"  - {r['path']} (GT: {r['gt']})")
        print(f"    Classical predicted: {r['c_pred']} ({r['c_confidence']:.3f})")
        print(f"    Quantum predicted:   {r['q_pred']} ({r['q_confidence']:.3f})")
    
    print("\n" + "="*30)
    print("  END OF REPORT")
    print("="*30)

In [14]:
run_evaluation()

Using device: cuda
Loading classical model from: outputs/resnet18_finetuned_with_head.pt
Loading quantum model from: artifacts/hybrid_qml_best.pt
Scanning for images in: final_test_full
Found 340 images to test.
Running predictions...

  EVALUATION REPORT

--- 1. Overall Accuracy ---
Classical: 307/340  (90.29%)
Quantum:   315/340  (92.65%)

--- 2. Average Confidence ---
  (Probability assigned to the *predicted* class)
Classical (Correct): 0.9787
Classical (Wrong):   0.8966
Quantum (Correct):   0.8082
Quantum (Wrong):     0.7201

--- 3. Average Probability Margin ---
  (Average distance from 0.5, a measure of decisiveness)
Classical: 0.4707
Quantum:   0.3017

--- 4. Discrepancy Report ---

Classical CORRECT, Quantum WRONG (10):
  - positive\123 (16).jpg (GT: positive)
    Classical predicted: positive (0.999)
    Quantum predicted:   negative (0.604)
  - positive\16.jpg (GT: positive)
    Classical predicted: positive (0.887)
    Quantum predicted:   negative (0.793)
  - positive\18.j